In [1]:
# Data processing script
# 
# assign working directory path
wd.path <- "~/in"
setwd(wd.path)
set.seed(1234)
library(dplyr)
library(data.table)            # to get fread
# library(foreach)
# library(caret)
# library(reshape2)
library(rms)
library(WeightedROC)

#Sample mode function from Source: https://stackoverflow.com/questions/2547402/is-there-a-built-in-function-for-finding-the-mode
Mode <- function(x, na.rm = FALSE) {
  if (na.rm) {
    x = x[!is.na(x)]
  }
  
  ux <- unique(x)
  return(ux[which.max(tabulate(match(x, ux)))])
}

imputationFunction <- function(imputeToData, imputeFromData, FUN, missingCols, suffix){
  # imputeToData -  Imputation to be done on this data
  # imputeFromData - Imputations calculation from this data
  # FUN - imputation function
  # missingCols - missing value column names
  # suffix - suffix to add after column name
  
  imputeToData <- imputeToData[, names(imputeToData) %in% missingCols]
  imputeFromData <- imputeFromData[, names(imputeFromData) %in% missingCols]
  imputeVec <- apply(imputeFromData, 2, function(x) FUN(x, na.rm = T))
  
  for (i in 1:length(missingCols)) {
    imputeToxData[is.na(imputeToData[, names(imputeToData) %in% missingCols[i]])
                 , names(imputeToData) %in% missingCols[i]] <- imputeVec[names(imputeVec) %in% missingCols[i]]
  }
  
  names(imputeToData) <- paste0(names(imputeToData), suffix)
  return(imputeToData)
}


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

Loading required package: Hmisc
Loading required package: lattice
Loading required package: survival
Loading required package: Formula
Loading required package: ggplot2

Attaching package: ‘Hmisc’

The following objects are masked from ‘package:dplyr’:

    combine, src, summarize

The following objects are masked from ‘package:base’:

    format.pval, round.POSIXt, trunc.POSIXt, units

Loading required package: SparseM

Attaching package: ‘SparseM’

The following object is masked from ‘package:base’:

    backsolve



### Get a small sample dataset to test the pipeline

In [43]:
samp_row = 1000                                                 # number of rows to take in the sample
num.impute = 1                                                  # number of imputations to perform on each node 
feat_list = c('SK_ID_CURR','TARGET','fold','Weights',
              
              
'Avg_EXT_SOURCE_application',
'ANNUITY_LENGTH',
'MAX_DAYS_CREDIT_bureau',
'CNT_INSTALMENT_FUTURE_min_sd_POS_CASH',
'MAX_DAYS_CREDIT_ENDDATE_bureau',
'CODE_GENDER',
'CREDIT_TO_GOODS_RATIO',
'EXT_SOURCE_1.x',
'EXT_SOURCE_2.x',
'EXT_SOURCE_3.x'              
)

requested_formula = ~ TARGET +Avg_EXT_SOURCE_application + ANNUITY_LENGTH + MAX_DAYS_CREDIT_bureau + 
    CNT_INSTALMENT_FUTURE_min_sd_POS_CASH + MAX_DAYS_CREDIT_ENDDATE_bureau + 
    CODE_GENDER + CREDIT_TO_GOODS_RATIO + EXT_SOURCE_1.x + EXT_SOURCE_2.x + 
    EXT_SOURCE_3.x

In [40]:
print(Sys.time())
train <- fread('train_modified.csv', nrows=samp_row) # applications test data
test <- fread('test_modified.csv', nrows=samp_row) # applications train data
print(Sys.time())

[1] "2018-08-28 22:56:50 UTC"
Read 1000 rows and 3205 (of 3205) columns from 4.639 GB file in 00:00:04
[1] "2018-08-28 22:57:47 UTC"


In [41]:
train_imp <- train[,feat_list,with=FALSE]
print(Sys.time())

test_imp <- test
test_imp$fold <- rep(NA,nrow(test_imp))
test_imp$Weights <- rep(NA,nrow(test_imp))
test_imp$TARGET <- rep(NA,nrow(test_imp))
test_imp <- test_imp[, feat_list, with=FALSE]
print(Sys.time())

M <- rbind(train_imp, test_imp)
M[M == 'XNA' | M == 'Unknown'] <- NA
M = droplevels(M)
print(Sys.time())

# replace Inf with NA as transcan, aregimpute cannot handle Inf
system.time(invisible(lapply(names(M),function(.name) set(M, which(is.infinite(M[[.name]])), j = .name,value =NA))))
print(Sys.time())

[1] "2018-08-28 22:57:47 UTC"
[1] "2018-08-28 22:57:48 UTC"
[1] "2018-08-28 22:57:48 UTC"


   user  system elapsed 
  0.006   0.001   0.007 

[1] "2018-08-28 22:57:48 UTC"


In [44]:
print(Sys.time())

library(parallel)
# Using all cores can slow down the computer
# significantly, I therefore try to leave one
# core alone in order to be able to do something 
# else during the time the code runs

# cores_2_use <- detectCores() - 1
cores_2_use <- 1

cl <- makeCluster(cores_2_use)
clusterSetRNGStream(cl, 9956)

clusterExport(cl, c("M", "num.impute", "requested_formula"))   # specify the list of objects to pass to each thread
clusterEvalQ(cl, library(rms))
imp_pars <- 
  parLapply(cl = cl, X = 1:cores_2_use, fun = function(no){
    mi <- aregImpute(requested_formula
                 , data=M, n.impute=num.impute, nk=4, pr=FALSE)
  })
stopCluster(cl)
print("Finished Imputation on all clusters.")
print(Sys.time())

setDF(M)             # converting the Data Table back to Data Frame, otherwise the join would not work 

completed.data = list()
for (i in 1:length(imp_pars)){
    for (j in 1:num.impute){
        imputed.data <- impute.transcan(imp_pars[[i]],imputation=j,data=M,list.out=TRUE,pr=FALSE,check=FALSE)
        k = (i-1)*num.impute + j
        completed.data[[k]] <- M
        completed.data[[k]][names(imputed.data)] <- imputed.data
        }
}
print("Finished Imputing the datasets")
print(Sys.time())

[1] "2018-08-28 22:59:41 UTC"


[[1]]
 [1] "rms"           "SparseM"       "Hmisc"         "ggplot2"      
 [5] "Formula"       "survival"      "lattice"       "RevoUtils"    
 [9] "stats"         "graphics"      "grDevices"     "utils"        
[13] "datasets"      "RevoUtilsMath" "methods"       "base"

[1] "Finished Imputation on all clusters."
[1] "2018-08-28 23:00:05 UTC"
[1] "Finished Imputing the datasets"
[1] "2018-08-28 23:00:05 UTC"


In [46]:
train <- completed.data[[1]][!is.na(completed.data[[1]]$Weights),]
test <- completed.data[[1]][is.na(completed.data[[1]]$Weights),]
string_formula <- TARGET ~ +rcs(Avg_EXT_SOURCE_application, 4) + rcs(ANNUITY_LENGTH, 
    4) + MAX_DAYS_CREDIT_bureau + rcs(CNT_INSTALMENT_FUTURE_min_sd_POS_CASH, 
    4) + rcs(MAX_DAYS_CREDIT_ENDDATE_bureau, 4) + CODE_GENDER + 
    rcs(CREDIT_TO_GOODS_RATIO, 4) + rcs(EXT_SOURCE_1.x, 4) + 
    rcs(EXT_SOURCE_2.x, 4) + rcs(EXT_SOURCE_3.x, 4)
requested_formula = as.formula(string_formula)

In [51]:
#Perform 5 fold cross validation
model = list()
mod_anova = list()
test_roc = list()

for(i in 1:5){
    testData <- train[train$fold==i,]
    trainData <- train[train$fold!=i,]
    

    a_model = lrm(data=trainData, formula = requested_formula, weight=Weights, tol=1E-8,
                  x=TRUE,y=TRUE)
        
    # diagnostic plots
    ## Variable Importance
    mod_anova[[i]] <- anova(a_model)
    
    ## Validation Statistics
    print(validate(a_model, B=10))
    model[[i]] <- a_model

    ## Calibration Curve
    # plot(calibrate(a_model, B=10))

    # use a fitted model to score a dataset, convert score to probability
    test_scored <- cbind(testData, predict(a_model, testData, se.fit=TRUE))
    test_scored$pred = exp(test_scored$linear.predictors)/(1+exp(test_scored$linear.predictors))

    # Weighted AUROC
    tp.fp <- WeightedROC(test_scored$pred,test_scored$TARGET,test_scored$Weights)
    test_roc[[i]] <- WeightedAUC(tp.fp)
    print(test_roc[[i]])

    # Unweighted AUROC
    tp.fp <- WeightedROC(test_scored$pred,test_scored$TARGET)
    print(WeightedAUC(tp.fp))
    
}

Warning message in lrm(data = trainData, formula = requested_formula, weight = Weights, :
“currently weights are ignored in model validation and bootstrapping lrm fits”

          index.orig training    test optimism index.corrected  n
Dxy           0.4753   0.7628  0.6288   0.1340          0.3413 10
R2            0.5861   0.3644  0.2102   0.1542          0.4319 10
Intercept     0.0000   0.0000 -0.6482   0.6482         -0.6482 10
Slope         1.0000   1.0000  0.6158   0.3842          0.6158 10
Emax          0.0000   0.0000  0.2492   0.2492          0.2492 10
D             0.0206   0.1563  0.0890   0.0673         -0.0467 10
U            -0.0025  -0.0025  0.0253  -0.0278          0.0253 10
Q             0.0230   0.1587  0.0637   0.0950         -0.0720 10
B             0.0811   0.0493  0.0610  -0.0117          0.0928 10
g             9.8827   3.1927  1.8595   1.3332          8.5495 10
gp            0.1210   0.1009  0.0832   0.0176          0.1034 10
[1] 0.3866744
[1] 0.5483333


Warning message in lrm(data = trainData, formula = requested_formula, weight = Weights, :
“currently weights are ignored in model validation and bootstrapping lrm fits”

          index.orig training    test optimism index.corrected  n
Dxy           0.4355   0.6866  0.5692   0.1174          0.3181 10
R2            0.3049   0.2944  0.1887   0.1057          0.1992 10
Intercept     0.0000   0.0000 -0.5528   0.5528         -0.5528 10
Slope         1.0000   1.0000  0.7320   0.2680          0.7320 10
Emax          0.0000   0.0000  0.1878   0.1878          0.1878 10
D             0.0051   0.1275  0.0783   0.0491         -0.0441 10
U            -0.0025  -0.0025  0.0084  -0.0110          0.0084 10
Q             0.0076   0.1300  0.0699   0.0601         -0.0525 10
B             0.0647   0.0545  0.0602  -0.0057          0.0704 10
g             3.2482   2.0341  1.4748   0.5593          2.6890 10
gp            0.0666   0.0955  0.0774   0.0180          0.0486 10
[1] 0.328137
[1] 0.7588462


Warning message in lrm(data = trainData, formula = requested_formula, weight = Weights, :
“currently weights are ignored in model validation and bootstrapping lrm fits”

          index.orig training    test optimism index.corrected  n
Dxy           0.3523   0.6428  0.5075   0.1353          0.2170 10
R2            0.4632   0.2597  0.1295   0.1302          0.3330 10
Intercept     0.0000   0.0000 -0.8376   0.8376         -0.8376 10
Slope         1.0000   1.0000  0.6183   0.3817          0.6183 10
Emax          0.0000   0.0000  0.2953   0.2953          0.2953 10
D             0.0155   0.1046  0.0494   0.0552         -0.0397 10
U            -0.0024  -0.0024  0.0177  -0.0202          0.0177 10
Q             0.0179   0.1070  0.0317   0.0754         -0.0574 10
B             0.0660   0.0503  0.0563  -0.0059          0.0720 10
g             3.3500   2.0526  1.2083   0.8442          2.5057 10
gp            0.0935   0.0822  0.0598   0.0224          0.0712 10
[1] 0.490098
[1] 0.5636364


Warning message in lrm(data = trainData, formula = requested_formula, weight = Weights, :
“currently weights are ignored in model validation and bootstrapping lrm fits”

          index.orig training    test optimism index.corrected  n
Dxy           0.2904   0.6625  0.5156   0.1469          0.1435 10
R2            0.4686   0.2720  0.1527   0.1193          0.3493 10
Intercept     0.0000   0.0000 -0.6815   0.6815         -0.6815 10
Slope         1.0000   1.0000  0.6776   0.3224          0.6776 10
Emax          0.0000   0.0000  0.2361   0.2361          0.2361 10
D             0.0170   0.1172  0.0625   0.0547         -0.0377 10
U            -0.0026  -0.0026  0.0126  -0.0152          0.0126 10
Q             0.0196   0.1197  0.0499   0.0699         -0.0503 10
B             0.0759   0.0563  0.0611  -0.0047          0.0806 10
g             4.1506   1.9142  1.2744   0.6399          3.5107 10
gp            0.0963   0.0930  0.0701   0.0228          0.0735 10
[1] 0.5767188
[1] 0.6138072


Warning message in lrm(data = trainData, formula = requested_formula, weight = Weights, :
“currently weights are ignored in model validation and bootstrapping lrm fits”

          index.orig training    test optimism index.corrected  n
Dxy           0.3964   0.7668  0.6238   0.1430          0.2534 10
R2            0.5276   0.3597  0.2091   0.1506          0.3770 10
Intercept     0.0000   0.0000 -0.6949   0.6949         -0.6949 10
Slope         1.0000   1.0000  0.6384   0.3616          0.6384 10
Emax          0.0000   0.0000  0.2526   0.2526          0.2526 10
D             0.0185   0.1489  0.0834   0.0655         -0.0469 10
U            -0.0025  -0.0025  0.0254  -0.0279          0.0254 10
Q             0.0210   0.1514  0.0580   0.0934         -0.0724 10
B             0.0772   0.0479  0.0546  -0.0067          0.0839 10
g             5.0370   2.9184  1.6843   1.2341          3.8029 10
gp            0.1269   0.0958  0.0764   0.0194          0.1075 10
[1] 0.4519207
[1] 0.5276989


## Experiments

1. Are the imputation from the different cores giving rise to the same datasets?

In [ ]:
library(parallel)
# Using all cores can slow down the computer
# significantly, I therefore try to leave one
# core alone in order to be able to do something 
# else during the time the code runs

#cores_2_use <- detectCores() - 1
cores_2_use <- 2
num.impute <- 2

cl <- makeCluster(cores_2_use)
clusterSetRNGStream(cl, 9956)

clusterExport(cl, c("M", "num.impute"))   # specify the list of objects to pass to each thread
clusterEvalQ(cl, library(rms))
imp_pars <- 
  parLapply(cl = cl, X = 1:cores_2_use, fun = function(no){
    mi <- aregImpute(~ TARGET +Avg_EXT_SOURCE_application + ANNUITY_LENGTH + MAX_DAYS_CREDIT_bureau + 
    CNT_INSTALMENT_FUTURE_min_sd_POS_CASH
                 , data=M, n.impute=num.impute, nk=4, pr=FALSE)
  })
stopCluster(cl)

completed.data = list()
for (i in 1:length(imp_pars)){
    for (j in 1:num.impute){
        imputed.data <- impute.transcan(imp_pars[[i]],imputation=j,data=M,list.out=TRUE,pr=FALSE,check=FALSE)
        k = (i-1)*num.impute + j
        completed.data[[k]] <- M
        completed.data[[k]][names(imputed.data)] <- imputed.data
        }
}

print("Variables that have any missing in completed.data.1")
names(completed.data[[1]])[apply(completed.data[[1]], 2, function(x) sum(is.na(x))) > 0]

print("Variables that have any missing in completed.data.2")
names(completed.data[[2]])[apply(completed.data[[2]], 2, function(x) sum(is.na(x))) > 0]

print("Are the two completions identical?")
identical(x=completed.data[[1]], y=completed.data[[2]])

print("Three sample imputed values for MAX_DAYS_CREDIT_bureau, imputation 1 on Core 1")
print(imp_pars[[1]]$imputed$MAX_DAYS_CREDIT_bureau[1:3,1])
print("Three sample imputed values for MAX_DAYS_CREDIT_bureau, imputation 1 on Core 2")
print(imp_pars[[2]]$imputed$MAX_DAYS_CREDIT_bureau[1:3,1])

# Useful Tools

In [ ]:
# Find out the memory usage of each item
sort(sapply(ls(), function(x) format(object.size(get(x)), unit = 'auto')))

In [42]:
# Mechanically create a formula for imputation

col_set = M %>% names()
col_set = col_set[!col_set %in% c("TARGET", 'SK_ID_CURR', 'fold', 'Weights')]

string_formula = "TARGET~"
for(i in col_set){
  if(class(M[[i]]) == "numeric"){
    tmp_component = paste0("+",i)
  }
  if(class(M[[i]]) != "numeric"){
    tmp_component = paste0("+",i)
  }
  string_formula = paste0(string_formula, tmp_component)  
}
requested_formula = as.formula(string_formula)
print(requested_formula)

TARGET ~ +Avg_EXT_SOURCE_application + ANNUITY_LENGTH + MAX_DAYS_CREDIT_bureau + 
    CNT_INSTALMENT_FUTURE_min_sd_POS_CASH + MAX_DAYS_CREDIT_ENDDATE_bureau + 
    CODE_GENDER + CREDIT_TO_GOODS_RATIO + EXT_SOURCE_1.x + EXT_SOURCE_2.x + 
    EXT_SOURCE_3.x


In [45]:
# Mechanically create a formula for imputation

col_set = M %>% names()
col_set = col_set[!col_set %in% c("TARGET", 'SK_ID_CURR', 'fold', 'Weights')]

string_formula = "TARGET~"
for(i in col_set){
  if(class(M[[i]]) == "numeric"){
    tmp_component = paste0("+rcs(",i,",4)")
  }
  if(class(M[[i]]) != "numeric"){
    tmp_component = paste0("+",i)
  }
  string_formula = paste0(string_formula, tmp_component)  
}
requested_formula = as.formula(string_formula)
print(requested_formula)

TARGET ~ +rcs(Avg_EXT_SOURCE_application, 4) + rcs(ANNUITY_LENGTH, 
    4) + MAX_DAYS_CREDIT_bureau + rcs(CNT_INSTALMENT_FUTURE_min_sd_POS_CASH, 
    4) + rcs(MAX_DAYS_CREDIT_ENDDATE_bureau, 4) + CODE_GENDER + 
    rcs(CREDIT_TO_GOODS_RATIO, 4) + rcs(EXT_SOURCE_1.x, 4) + 
    rcs(EXT_SOURCE_2.x, 4) + rcs(EXT_SOURCE_3.x, 4)


In [ ]:
# replace Inf with NA as transcan, aregimpute cannot handle Inf
system.time(invisible(lapply(names(M),function(.name) set(M, which(is.infinite(M[[.name]])), j = .name,value =NA))))

## Conclusions / Takeaways

1. transcan and aregImpute cannot handle infinite values, need to first replace with NA. Use the following code.
> system.time(invisible(lapply(names(M),function(.name) set(M, which(is.infinite(M[[.name]])), j = .name,value =NA))))
    
2. aregImpute more fault-tolerant than transcan, latter would quit if it does not converge in 50 iterations.
> Error in transcan(~Avg_EXT_SOURCE_application + ANNUITY_LENGTH + MAX_DAYS_CREDIT_bureau + : no convergence in 50 iterations

3. data.table::fread much better than read.csv for larger datasets, 2.5min vs 40min!

4. Peek at the source code for R packages to understand how to use the dataset! (https://github.com/harrelfe/Hmisc/blob/master/R/fit.mult.impute.s)

5. identical and all.equal can be used to check if two data frames are the same (https://stackoverflow.com/questions/10978895/how-to-compare-two-dataframes)

6. fread and read.csv have different behaviors:
   1. V1 vs X for the unnamed first column
   2. train_imp[,c(1,3:ncol(train_imp),2)] would not reorder the column if read in by fread
   3. DT[,feat_list] would look for a column named feat_list, need to use with=FALSE to use it as Data Frame
   Check 1.1 in the FAQ
   http://datatable.r-forge.r-project.org/datatable-faq.pdf
   4. Need to convert Data Table back to Data Frame (using setDF(df)) before using the old-fashioned join, otherwise would get the following error:
   > Error in `[.data.table`(x, i, which = TRUE): When i is a data.table (or character vector), the columns to join by must be specified either using 'on=' argument (see ?data.table) or by keying x (i.e. sorted, and, marked as sorted, see ?setkey). Keyed joins might have further speed benefits on very large data due to x being sorted in RAM.
   
7. R codes could be difficult to debug, could just tell you there is an error, but not the offending variables., e.g.:
   1. NAs are not allowed in subscripted assignments???
   
8. After Multiple Imputation, my model seems way over-fitted, need to figure out the correct way to use it, as it doesn't seem as straightforward as I had originally expected.